# Problem Introduction

Nowaday, people seem to be more accustomed to typing by hand, whether at work or in life, rather than writing by hand. Compared to the pen and paper, the 26-key input method clearly occupies a larger part of the lives of many people. As a result, the previous typos have become errors in the document. How to correctly identify these errors and correct them intelligently becomes a feature we urgently need. Now our commonly used document editing software has introduced the function of typo self-correction. For example, when you type "teh", "Word" will automatically change it to "the". When you type "speling" with Google search, in less than 0.1 seconds, Google will return: What you are looking for is "spelling". (Yahoo! and Microsoft have similar features). Of course, this is the simplest example. How can a computer intelligently find a document error and correct it under normal circumstances ?

In fact, what we usually call a typo can be broken down into two kinds of mistakes.

1. None-word error
That is, the wrongly written words are not the existing words in the dictionary, such as creation written as creaton. This kind of error is very well recognized, just need to have a pre-loaded dictionary.

2. Real-word error
As the name implies, this kind of error refers to the existence of the wrong word in the dictionary. Relatively speaking, this kind of error is more difficult to identify. Simple examples such as the result of losing three.

We will describe how to make automatic corrections after the wrong words are identified.

# Working principle

Given a word, our task is to choose the correct spelling word that is most similar to it. (If the word itself is spelled correctly, then the closest thing is it.) Of course, it is impossible to find similar words absolutely. For example, given the word "laters", should it be corrected for "late" or "latest"? These difficulties indicate that we need to use probability theory instead of rule-based judgment. We say, given a word w, we want to find a correct word c in all correct words database, so that the conditional probability for w is the largest, that is:

$argmax_{c∈candidates} \,\, p(c|w)$

According to Bayesian theory, the above equation is equivalent to:

$argmax_{c∈candidates}  \,\, \frac{P(c)×P(w|c)}{P(w)}$

Since $P(w)$ is the same for each c to be selected, we ignore this factor and the final formula is transformed into:

$argmax_{c∈candidates}  \,\, P(c)×P(w|c)$

There are four main parts in this formula:

1. Selection agency: $argmax$ <br>
    We choose the most probable word in the alternative word as the output
2. Alternative model: $c∈candidates$ <br>
    This section tells us which words to consider as alternatives.
3. Language model: $P(c)$ <br>
    The probability that the word c appears in the corpus. For example, in an English corpus, 7% of the words are "the", then P(the)=0.07
4. Error model P(w|c) <br>
When the user wants to input c, the probability of inputting as w is wrong. For example, P(teh|the) should be much larger than P(theeexyz|the).

We replace the delay probability $P(c|w)$ with the conditional probability $P(w|c)$ and the prior probability $P(c)$, which are easy to consider and solve, so that the problem is easier to analyze and solve.

# Algorithm Implementation

The first is to calculate $P(c)$. We can read in a huge text file, [big.txt](http://www.norvig.com/big.txt), which has about a few million words (equivalent to a corpus). This file is composed of some books that can be obtained from the Gutenberg project, [Wiktionary](https://en.wiktionary.org/wiki/Wiktionary:Frequency_lists) and [British National Corpus](http://www.kilgarriff.co.uk/bnc-readme.html) corpus.

In [1]:
import numpy as np
# count the number of each word
from collections import Counter
from nltk.corpus import gutenberg
# natural language processing package
import nltk
import collections
import re

def load_words(file_name):
    word = []
    with open(file_name, 'r') as f:
        for line in f:
            # split words by utilizing nltk.word_tokenize() function
            word += nltk.word_tokenize(line)
    return word

#download gutenberg corpus
nltk.download('gutenberg')

# obtain Corpus for priori probability c
Corpus = load_words("big.txt") + gutenberg.words()

# it's a dictionary and obtain the number of each word
wordDict = Counter(Corpus)

# obtain the words set including all of words, use for checking 
# whether a word is in the set or not.
wordSet = set(wordDict.keys())

print(len(Corpus))

[nltk_data] Downloading package gutenberg to
[nltk_data]     /Users/revincent/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


3911095


In fact, wordSet stores how many times each word appears in the corpus. But one question is what to do if you encounter a new word we have never seen before. If a word is spelled exactly right, but the word is not included in the corpus, the word will never appear in the training set. So, we have to return the probability that the word appears to be 0. This situation is not very good, because the probability 0 means that this event is absolutely impossible. In our probability model, we expect to represent this situation with a small probability. In fact, there are many standard methods for forming this problem. We choose one of the most simple method: New words that have never been seen are assumed to have occurred once. This process generally becomes "smoothing", because we set the probability distribution to 0 to a small probability value.

Then the question is: Given a word w, how can I enumerate all possible correct spellings? In fact, the predecessors have studied very well, this is a concept of "editing distance". Editing distance between two words: defined as the use of insertion (insert a single letter in a word), deletion (delete a single letter), transposition (exchange two adjacent letters), substitution (change one letter to another) to form another word.
The following function returns all collections with a editing distance of 1 from the word w.

In [2]:
def edit_distance_1(word):
    # candidate alphabet
    alphabet = 'abcdefghijklmnopqrstuvwxyz'
    n = len(word)
    
    # delete operation
    deletion = []
    for i in range(n):
        # delete the ith alphabet in word
        deletion.append(word[0:i] + word[i + 1:])
        
    # replace operation
    substitution = []
    for i in range(n):
        for c in alphabet:
            # use alphabet c to replace the ith alphabet in word
            substitution.append(word[0:i] + c + word[i+1:])
            
    # insert operation
    insertion = []
    for i in range(n+1):
        for c in alphabet:
            # insert alphabet c to the ith position in word
            insertion.append(word[0:i] + c + word[i:])
            
    # exchange operation
    transposition = []
    for i in range(n-1):
        # exchange ith and (i+1)th position
        transposition.append(word[0:i] + word[i+1] + word[i] + word[i+2:])
    
    # delete repeation
    edit1_words = set(deletion + substitution + insertion + transposition)
    
    return edit1_words

# print(len(edit_distance_1("something")))

Obviously, this collection is very big. For a word of length n, there may be n deletions, n-1 swaps, 26n substitutions, and 26(n+1) insertions.(There is a bit of repetition). For example, there are 511 words which generated by "something" with editing distance of 1, but it is actually 494.

However, we can define a module that identifies the correctness of these generated alternative words, matching only the words that exist in the dictionary. This set will become very small, because many of the randomly generated words are illegally spelled and not really exist.

In [3]:
def known(words):
    
    # Before optimization
    '''
    known_words = []
    for w in words:
        # if w in Corpus(wordSet), that means w is known word, else is unknown word.
        if w in wordSet:
            known_words.append(w)
    
    return set(known_words)
    '''
    
    # After optimization
    # Use the intersection operation of sets with python to accelerate
    
    return words & wordSet

# print(len(known(edit_distance_1("something"))))

The spell-checking literature claims that approximately 80-95% of spelling errors are within editing distance of 1. However, when we experimented with a 270 misspelled corpus, we found that only 76% of misspellings belonged to a set with an editing distance of 1. So we started thinking about editing those words with a distance of 2.

In [4]:
def edit_distance_2(word):
    
    edit2_words = []
    # obtain editing distance of 1 and known words, reduce the time complexity
    # and then use edit_distance_1() function editing again, will obtain 
    # editing distance of 2.
    edit1_words = known(edit_distance_1(word))
    for e1 in edit1_words:
        edit2 = known(edit_distance_1(e1))
        edit2_words += edit2
    
    # delete repeation
    
    return set(edit2_words)

print(len(edit_distance_2("something")))

3


The last thing left is the error model part $P(w|c)$. Choose a simple method:
1. Editing the correct word with a distance of 1 is higher than the editing distance of 2,
2. The correct word priority with an edit distance of 0 is higher than the edit distance of 1.
<br>Therefore, writing it out in code is:

In [26]:
# according to priority definition to select candidate words
def candidate_word(word):
    
    candidates = known(set([word])) or known(edit_distance_1(word)) \
    or known(edit_distance_2(word)) or set([word])
    
    return candidates

# according to the candidate words, select the maximum probability as correct word
def correct_word(word):
    
    # The number of occurrences of a word as a basis for sorting
#     print(candidate_word(word))
    return max(candidate_word(word), key=lambda w: wordDict[w])

print(correct_word("gruop"))
print(correct_word("teh"))

# will output "the"

group
the


# Evaluation

## Performance
Roger Mitton's Birkbeck misspelling corpus was downloaded from the [Oxford Text Archive](http://ota.ahds.ac.uk/texts/0643.html). There are 2 misspelling sets to be used for testing the accuracy of the model and time cost. 

In [6]:
import time
import os
import re

# check if the word contains illegal characters
def isvalid(word): 
    
    '''include numbers'''
    if bool(re.search(r'\d', word)) == True:
        return False
    
    '''inclue - '''
    if bool("-" in word) == True:
        return False
    
    '''include * '''
    if bool("*" in word) == True:
        return False
    
    '''include ' ''' 
    if bool("'" in word) == True:
        return False
    
    return True

def load_spellerrors1(file_name):
    word_dict = {}
    with open(file_name) as fr:
        for line in fr:
            splited = line.strip(" ").strip("\n").split(":")
            correct, wrong = splited[0], [x.strip(" ") for x in splited[1].split(" ")]
            word_dict[correct] = wrong[1:]
    
    return word_dict

def load_spellerrors2(file_name):
    word_dict = {}
    total_line = 0
    with open(file_name) as fr:
        for line in fr:
            # due to autoencorrect.spell() spend too much time to correct, 
            # only choose 200 lines data
            if total_line > 200:
                break
            total_line += 1
            splited = line.strip(" ").strip("\n").split(":")
            correct, wrong = splited[0], [x.strip(" ") for x in splited[1].split(",")]
            if isvalid(correct) == False:
                continue
            word_dict[correct] = []
            for w in wrong:
                if isvalid(w) == True:
                    word_dict[correct].append(w)
#             print(correct, word_dict[correct])
    
    return word_dict

words_1 = load_spellerrors1("spell-errors1.txt")
words_2 = load_spellerrors2("spell-errors.txt")


In [13]:
def test_accuracy(word_dict):
    
    begin_time = time.time()
    correct_num = 0
    unknown_num = 0
    total_num = 0
    for correct, wrong in word_dict.items():
        for word in wrong:
#             print(correct, wrong)
            total_num += 1
            
            # if the word is unknown
            if correct not in wordSet:
                unknown_num += 1
                continue
                
            target = correct_word(word)
            if target == correct:
                correct_num += 1
                
    accuracy = correct_num * 1.0 / total_num
    end_time = time.time()
    spend_time = end_time - begin_time
    
    print("correct words: ", correct_num)
    print("unknown words: ", unknown_num)
    print("total word: ", total_num)
    print("spend time: ", spend_time, "seconds")
    print("accuracy: ", accuracy)


In [14]:
test_accuracy(words_1)

correct words:  164
unknown words:  10
total word:  270
spend time:  0.06961774826049805 seconds
accuracy:  0.6074074074074074


In [15]:
test_accuracy(words_2)

correct words:  194
unknown words:  35
total word:  823
spend time:  0.22311997413635254 seconds
accuracy:  0.23572296476306198


## Comparative Study

[Autocorrect](https://pypi.org/project/autocorrect/0.1.0/) is an open source library for Python that automatically corrects spelling errors in toolkits. We use the functions in this package to test the test set and compare the results with our model.

In [19]:
from autocorrect import spell
import warnings

warnings.filterwarnings('ignore')

def auto_correct(word_dict):
    
    begin_time = time.time()
    correct_num = 0
#     unknown_num = 0
    total_num = 0
    for correct, wrong in word_dict.items():
        for word in wrong:
            total_num += 1
            
            # if the word is unknown
#             if correct not in wordSet:
#                 unknown_num += 1
#                 continue
                
            target = spell(word)
            if target == correct:
                correct_num += 1
                
    accuracy = correct_num * 1.0 / total_num
    end_time = time.time()
    spend_time = end_time - begin_time
    
    print("correct words: ", correct_num)
#     print("unknown words: ", unknown_num)
    print("total word: ", total_num)
    print("spend time: ", spend_time, "seconds")
    print("accuracy: ", accuracy)


In [20]:
auto_correct(words_1)

correct words:  190
total word:  270
spend time:  5.1764140129089355 seconds
accuracy:  0.7037037037037037


In [21]:
auto_correct(words_2)

correct words:  267
total word:  823
spend time:  50.30043029785156 seconds
accuracy:  0.3244228432563791


## Conclusion

The accuracy and time overhead of our model on two different test sets respectively are 60.7%, 23.6% and 0.07s, 0.22s. The accuracy and time overhead of using python open source function "autocorrect.spell" on two different test sets respectively are 70.1%, 32.4% and 5.18s, 50.30s. Although the accuracy of "autocorrect.spell" is higher than proposed model with 9.4% and 8.8% respectively, the time it takes are 74 and 228 times in two different datasets. This shows that the proposed model is effective to some conditions.

# Proposed  Improvements

How can we achieve better results ? Let us look back at the three factors in the probability model: (1) $P(c)$; (2) $P(w|c)$; and (3) $argmax_{c}$. The examples of the program giving the wrong answer start, look at these three factors, what we still ignore ?

1. $P(c)$. There are two problems that cause the final misidentification. One of the most serious factors is the unknown word. In above two datasets, there are a total of 10 and 35 unknown words. These unknown words account for about 3.7% and 4% respectively. This problem can be solved by adding more corpus. The second factor that can lead to errors is that both words appear in our dictionary, but the one with the highest probability of our selection is not what the user wants.

2. $P(w|c)$. The second question is the priority order of the candidate words. So far, we have used a very simple model: the shorter the distance, the greater the probability.

3. When we correct words, we don’t consider the contextual relationship. Because in many cases, it is difficult to make decisions based solely on the word itself. The word itself is in the dictionary, but in the context it should be corrected to another word.